In [2]:
!pip install langchain text2vec

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [3]:
import json
with open('workflow_asset/catogory_sample.json', 'r') as f:
    category = json.loads(f.read())
    
with open('workflow_asset/catogory_sample_generate_llama270b.json', 'r') as f:
    category_generate = json.loads(f.read())

In [4]:
import pandas as pd

In [5]:
def generate_sample():
    
    for c, v in category.items():
        for product_sample in v:
            yield c, product_sample

        for product_sample in category_generate[c]:
            yield c, product_sample

In [6]:
df = pd.DataFrame(generate_sample(), columns=['category', 'sample'])
df['cs'] = df['category']+'-'+df['sample']

In [7]:
from langchain.vectorstores import FAISS, Chroma

In [8]:
!pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [9]:
from text2vec import SentenceModel
model = SentenceModel('shibing624/text2vec-base-chinese')
embeddings = model.encode(df['sample'])

2023-08-09 16:29:15.438 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


In [10]:
# from langchain.vectorstores.utils import DistanceStrategy
# from langchain.schema import Document
# docs = list(map(lambda x: Document(page_content=x[1]['cs'], metadata=x[1]), df.iterrows()))
# vdb = FAISS.from_embeddings(docs, embeddings, normalize_L2=True, distance_strategy=DistanceStrategy.COSINE)

In [11]:
from typing import Any, Dict, List, Optional
from text2vec import SentenceModel

class EmbeddingModel():
    def __init__(self, model='shibing624/text2vec-base-chinese', **kwargs):

        self.embedding_model = SentenceModel()


    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Compute doc embeddings using a Bedrock model.

        Args:
            texts: The list of texts to embed.

        Returns:
            List of embeddings, one for each text.
        """
        return self.embedding_model.encode(texts)

    def embed_query(self, text: str) -> List[float]:
        """Compute query embeddings using a Bedrock model.

        Args:
            text: The text to embed.

        Returns:
            Embeddings for the text.
        """
        return self.embedding_model.encode([text])[0]


In [12]:
from langchain.schema import Document
from langchain.vectorstores.utils import DistanceStrategy

In [13]:
match_col = 'sample' # cs, sample, category
docs = list(map(lambda x: Document(page_content=x[1][match_col], metadata=x[1]), df.iterrows()))
vdb = FAISS.from_documents(docs, EmbeddingModel(), normalize_L2=False, distance_strategy=DistanceStrategy.COSINE)

2023-08-09 16:29:29.254 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


In [14]:
df_samples = pd.read_csv('data/samples.csv', index_col=0)
df_samples = df_samples[df_samples['item_name'].apply(lambda x: '外送費' not in x)]
df_samples = df_samples[df_samples['item_name'].apply(lambda x: '平台費' not in x)]
df_samples = df_samples[df_samples['category'].apply(lambda x: '其他' not in x)]

In [15]:
item = df_samples.sample(1).iloc[0]
item

item_name    現貨+預購 貝恩 Baan NEW嬰兒保養柔濕巾80抽×24 
item_type                              日用百貨類
category                              家庭清潔洗劑
Name: 950792, dtype: object

In [16]:
item_name = item['item_name']
item_name

'現貨+預購 貝恩 Baan NEW嬰兒保養柔濕巾80抽×24 '

In [17]:
vdb.similarity_search_with_score(item_name, k=10)

[(Document(page_content='【Maybelline】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', metadata={'category': '衣物清潔', 'sample': '【Maybelline】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', 'cs': '衣物清潔-【Maybelline】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)'}),
  203.81702),
 (Document(page_content='悅嬰牛乳保久乳190mlx6入', metadata={'category': '保久乳', 'sample': '悅嬰牛乳保久乳190mlx6入', 'cs': '保久乳-悅嬰牛乳保久乳190mlx6入'}),
  205.97673),
 (Document(page_content='【Sunsilk】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', metadata={'category': '衣物清潔', 'sample': '【Sunsilk】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', 'cs': '衣物清潔-【Sunsilk】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)'}),
  211.9634),
 (Document(page_content='【Garnier】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', metadata={'category': '衣物清潔', 'sample': '【Garnier】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', 'cs': '衣物清潔-【Garnier】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)'}),
  212.53888),
 (Document(page_content='【Mary Kay】柔軟護衣精750ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', metadata={'category': '衣物清潔', 'sample': '【Mary Kay】柔軟護衣精750ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', 'cs': '衣物清潔-【Mary Kay】柔軟護衣精750ML(玫瑰/茉莉/山茶花/小蒼

In [18]:
from multiprocessing import Pool

In [19]:
p = Pool(8)

Process ForkPoolWorker-2:
Process ForkPoolWorker-4:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-7:
Process ForkPoolWorker-8:
Process ForkPoolWorker-3:
Process ForkPoolWorker-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **s

In [20]:
vdb.similarity_search_with_score([item_name, item_name], k=1)#[0][0].metadata['category']

[(Document(page_content='【Maybelline】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', metadata={'category': '衣物清潔', 'sample': '【Maybelline】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)', 'cs': '衣物清潔-【Maybelline】柔軟護衣精700ML(玫瑰/茉莉/山茶花/小蒼蘭/藍風鈴)'}),
  219.36035)]

In [21]:
match_col = 'category' # cs, sample, category
docs = list(map(lambda x: Document(page_content=x[1][match_col], metadata=x[1]), df.iterrows()))
vdb = FAISS.from_documents(docs, EmbeddingModel(), normalize_L2=False, distance_strategy=DistanceStrategy.COSINE)

2023-08-09 16:30:05.030 | DEBUG    | text2vec.sentence_model:__init__:76 - Use device: cpu


In [22]:
def classify(item_name):
    # return vdb.similarity_search_with_score(item_name, k=1)[0][0].metadata['category']
    return vdb.similarity_search(item_name, k=1)[0].metadata['category']

In [25]:
docs[0]

Document(page_content='現做咖啡飲品', metadata={'category': '現做咖啡飲品', 'sample': '全家特濃拿鐵', 'cs': '現做咖啡飲品-全家特濃拿鐵'})

In [26]:
test = df_samples.sample(10000)
category

{'現做咖啡飲品': ['全家特濃拿鐵', '西西里風味檸檬氣泡咖啡'],
 '罐裝/瓶裝咖啡': ['《UCC》無糖咖啡飲料185g', 'SANGARIA 圓潤咖啡飲料-拿鐵 (280gx24入)'],
 '沖泡式咖啡': ['西雅圖即品拿鐵無加糖2合1咖啡21g', '【King Coffee 王者咖啡】經典風味推薦組：三合一滴滴煉奶即溶咖啡'],
 '濾掛式咖啡': ['【熙舍咖啡】義大利濾掛式咖啡綜合風味10g/包', '《西雅圖》極品濾掛咖啡-曼巴風情(50入/盒)'],
 '即溶咖啡': ['AGFMAXIM咖啡罐(藍)-奢華摩登 (80g)', 'AGFMAXIM咖啡罐(藍)-奢華摩登 (80g)'],
 '咖啡相關食用品': ['哥倫比亞咖啡豆(Supremo等級)', '冠軍單品咖啡豆 - 200g'],
 '現做茶飲品': ['桂花四季春', '春芽冷露(中) + 白杏'],
 '茶飲': ['【分批取】義美錫蘭紅茶946ml', '泰山冰鎮大麥綠茶300ml*6入'],
 '碳酸/汽水': ['金車奧利多水寡糖飲料2000ml', '巨峰葡萄氣泡飲'],
 '果汁/果醋飲': ['innocent100%椰子水1000ml', '蜂蜜藍莓醋'],
 '運動飲料': ['【分批取】黑松FIN補給飲料PET580ml', '寶礦力水得580ml'],
 '養生飲品': ['薑汁奶茶 熱', '【四喜】１８罐—冰糖蓮子白木耳露500ml±3%/罐'],
 '能量飲料': ['魔爪超越碳酸能量飲料355ml', '維他露P330ml (24入)'],
 '特殊風味飲品': ['金車健酪YoGoFresh乳酸飲料-原味580ml', '波蜜果菜汁250mlx6入'],
 '沖泡類飲品': ['【分批取】金品茗茶厚韻烏龍茶三角立體茶包2.5gx22入', '【義美生機】台灣黑豆茶'],
 '現調果汁': ['芭樂檸檬 (店長推薦★)', '百香芭樂/中'],
 '礦泉水': ['台鹽海洋鹼性離子水', '【悅氏】礦泉水 600mlx24瓶'],
 '氣泡水': ['泰山Cheers Bee 蜂蜜氣泡水 590ml', '金車 CrystalValley礦沛氣泡水 585ml'],
 '機能水': ['BLACK POWER黑水運動機能礦泉水', '悅氏鎂

In [27]:
test

,item_name,item_type,category
1444620,原味蛋撻2顆 2023-05-31 b7jy-siky,點心/零食類,肉乾/肉紙
1036859,分格餐盤 分隔餐盤【附發票】便當盒 餐盤 餐盒 不鏽鋼餐盤,日用百貨類,餐廚用品
922692,蓮蓬頭支架🌞元氣小鋪🌞淋浴頭支架 花灑支架 蓮蓬頭掛架,家電類,生活家電
167411,拭拭樂 加厚嬰兒超純水柔濕紙巾80抽x24包-便利保濕蓋-箱,洗浴清潔/保養類,身體清潔/保養
503987,8ed2832efde30c8c,3C類,手機/通訊/週邊
...,...,...,...
84242,Queen's Eyelashes假睫毛纖長,洗浴清潔/保養類,彩妝/卸妝
872114,濟生睛寶沖洗液500ml,家電類,生活家電
1274640,9789572644546鬼滅之刃 幸福之花 全,書本/文教用品類,書籍
1207603,玉子舒肥嫩雞絲湯麵套餐Chicken Soup With N,生鮮食品類,冷藏調理


In [28]:
%%timeit
classify(test['item_name'].iloc[0])

40.9 ms ± 790 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
test['predict_category'] = p.map(classify, test['item_name']) #test['item_name'].apply(classify)

Process ForkPoolWorker-10:
Process ForkPoolWorker-9:
Process ForkPoolWorker-11:
Process ForkPoolWorker-12:
Process ForkPoolWorker-14:
Process ForkPoolWorker-13:
Process ForkPoolWorker-16:
Process ForkPoolWorker-15:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):

KeyboardInterrupt

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/p

In [ ]:
1

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, multilabel_confusion_matrix

In [ ]:
accuracy_score(test['category'], test['predict'])

In [ ]:
accuracy_score(test['category'], test['predict_cs'])

In [ ]:
sum(test['predict'] != test['predict_cs'])

In [ ]:
classify('無印化妝盒')

In [ ]:
test.sample(100)